## Todo List
- [x] Checar se formato de datas conferem.
- [x] Remover UGs sem liberação OC
- [x] Considerar coluna montagem. Onde estiver vazio, pegar valor de Torres para EOL, no skate.
- [ ] Separar um cojunto aleatório para teste
- [x] Calcular tempos de construção por marco: (IO_realizado - realizado), (CC_realizado - realizado) ...etc  (ordem:IO,CC,ME,OT)
- [x] Calcular atraso da operação comercial (realizado - outorgado)
- [x] Calcular média e dp do tempo de construção.
- [ ] Calcular data dos marcos medios (outorgado - tempo_medio_de_construcao) por fonte.
- [ ] Calcular atrasos com base nos marcos medios
- [ ] Calcular media e dp dos atrasos. 
- [ ] Normalizar os atrasos calculados (atraso - atraso_medio)/dp
- [ ] Construir matriz de correlação de Pearson
- [ ] Calcular os coeficientes para regressão linear
- [ ] Para UHE pegar todas UGs, para as demais pegar a primeira UG apenas 
- [ ] Comparar pegando primeira UG e media por grupo



In [1]:
import pandas as pd
import numpy as np
import re
import pyodbc
from pprint import pprint
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy import stats
from itertools import combinations

In [2]:
def outliers(df:pd.DataFrame,cols : list):
    index_outliers =  []
    for col in cols:
        s = df[col]
        Q1 = s.quantile(0.1)
        Q3 = s.quantile(0.9)
        lower_bound =  Q1
        upper_bound =  Q3 
        index_outliers.extend(s.index[(s < lower_bound) | (s > upper_bound)])
    return set(index_outliers)

In [3]:
bd_combined = pd.read_parquet("bd_combined.gzip")
bd_combined.index.name = "idx"
bd_combined.IdeUsinaOutorga = bd_combined.IdeUsinaOutorga.astype(str)
mask = (bd_combined.SigTipoGeracao == "UHE") & (bd_combined.IdeUsinaOutorga.duplicated(keep=False))
bd_combined.loc[mask,"IdeUsinaOutorga"] = bd_combined.loc[mask,"IdeUsinaOutorga"] +"_"+ bd_combined.loc[mask].groupby("IdeUsinaOutorga").cumcount().astype(str)

In [4]:
bd_combined = bd_combined[bd_combined.DatUGInicioOpComerOutorgado.notna()
                            & bd_combined.DatLiberOpComerRealizado.notna()].copy(deep=True)

In [5]:
nomes_tempo_construcao = {
    "IO" : ["DatLiberOpComerRealizado","DatInicioObraRealizado"],
    "CC" : ["DatLiberOpComerRealizado","DatConcretagemRealizado"],
    "ME" : ["DatLiberOpComerRealizado","DatMontagemRealizado"],
    "MT" : ["DatLiberOpComerRealizado","DatConclusaoTorresRealizado"],
    "OT" : ["DatLiberOpComerRealizado","DatLiberacaoSFGTeste"],} 

tempo_construcao_por_UG = bd_combined[["IdeUsinaOutorga","SigTipoGeracao"]].copy(deep=True)
for key,values in nomes_tempo_construcao.items():
    tempo_construcao_por_UG[key] = bd_combined[values[0]]-  bd_combined[values[1]]

tempo_construcao_por_UG["Atraso"]  =  bd_combined["DatLiberOpComerRealizado"]-  bd_combined["DatUGInicioOpComerOutorgado"] 
tempo_construcao_por_UG.sample(10)

,IdeUsinaOutorga,SigTipoGeracao,IO,CC,ME,MT,OT,Atraso
idx,,,,,,,,
1324,30914,EOL,373 days,341 days,232 days,232 days,33 days,-22 days
2363,2092,PCH,-16530 days,-16579 days,-16939 days,NaT,0 days,-3 days
1348,30961,PCH,625 days,513 days,322 days,NaT,14 days,49 days
4271,38318,EOL,423 days,337 days,131 days,3 days,47 days,-872 days
2174,34387,UFV,487 days,NaT,268 days,NaT,62 days,407 days
1998,33469,UFV,212 days,NaT,151 days,NaT,28 days,-399 days
2083,33684,EOL,275 days,245 days,109 days,109 days,43 days,-121 days
1992,33380,UTE,432 days,NaT,432 days,NaT,40 days,324 days
256,28672,PCH,NaT,677 days,555 days,NaT,189 days,1756 days


In [16]:
is_first = True 
combinacoes = list(combinations([x for x in tempo_construcao_por_UG.columns if x not in ["IdeUsinaOutorga","SigTipoGeracao","Atraso"]],2))
#combinacoes = list(combinations(["IO","CC","ME","MT","OT"],2))

for p_marco, s_marco in combinacoes:
    idx_loop =  ((tempo_construcao_por_UG[p_marco] >= tempo_construcao_por_UG[s_marco]) | 
                 tempo_construcao_por_UG[p_marco].isna() | 
                 tempo_construcao_por_UG[s_marco].isna())

    idx = idx_loop if is_first else (idx & idx_loop)
    is_first = False 

tempo_construcao_por_UG_filtered = tempo_construcao_por_UG[idx & (tempo_construcao_por_UG_filtered.loc[tempo_construcao_por_UG_filtered.IO.isna())].copy(deep=True)
tempo_construcao_por_UG_filtered

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (Temp/ipykernel_16252/3688264923.py, line 13)

,IdeUsinaOutorga,SigTipoGeracao,IO,CC,ME,MT,OT,Atraso
idx,,,,,,,,
20,775,PCH,NaT,NaT,NaT,NaT,70 days,73 days
23,868,CGH,NaT,NaT,NaT,NaT,23 days,1438 days
32,1405,UTE,NaT,NaT,NaT,NaT,15 days,90 days
64,2707,CGH,NaT,NaT,NaT,NaT,102 days,1285 days
69,2958,UTE,NaT,NaT,NaT,NaT,33 days,-308 days
...,...,...,...,...,...,...,...,...
2512,30196,UTE,NaT,NaT,NaT,NaT,NaT,0 days
2513,30196,UTE,NaT,NaT,NaT,NaT,NaT,0 days
2517,30264,UTE,NaT,NaT,NaT,NaT,NaT,0 days


In [10]:
# Atrasos por usinas são calculados com base UG com menor atraso.
idx_primeira_ug = tempo_construcao_por_UG_filtered.groupby(["IdeUsinaOutorga"],sort=False).IO.idxmin()
tempo_construcao_por_usina = tempo_construcao_por_UG_filtered.loc[idx_primeira_ug]
#tempo_construcao_por_usina.set_index(keys = "IdeUsinaOutorga",inplace=True)
tempo_construcao_por_usina

ValueError: attempt to get argmin of an empty sequence

In [14]:
idx_primeira_ug = tempo_construcao_por_UG_filtered.groupby(["IdeUsinaOutorga"],sort=False).Atraso.idxmin()
tempo_construcao_por_usina = tempo_construcao_por_UG_filtered.loc[idx_primeira_ug]

In [ ]:
# Cria tabela para calculos por geracao
calculos_por_geracao = pd.DataFrame()
calculos_por_geracao["SigTipoGeracao"] = bd_combined.SigTipoGeracao.unique() 


# Realiza calculos por geracao
for coluna in tempo_construcao_por_usina.columns:
    if coluna not in ["IdeUsinaOutorga","SigTipoGeracao"]:
        calculos_por_geracao[f"{coluna}_media"] = calculos_por_geracao.SigTipoGeracao.apply(lambda x : tempo_construcao_por_usina[tempo_construcao_por_usina.SigTipoGeracao==x][coluna].mean())
        calculos_por_geracao[f"{coluna}_std"] = calculos_por_geracao.SigTipoGeracao.apply(lambda x : tempo_construcao_por_usina[tempo_construcao_por_usina.SigTipoGeracao==x][coluna].std())
display(calculos_por_geracao.set_index("SigTipoGeracao").T)

In [ ]:
calculos_por_geracao

In [ ]:
plot_created = False

for k,origem in enumerate(["skate","bdworgs","Ambos"]):





    if not plot_created:    
        num_subplots = 6*(len(colunas_atrasos_medio)-1)
        fig, axes = plt.subplots(num_subplots,3,figsize=(24,num_subplots*5))
        plot_created =True
    count = -1
    for i,SigTipoGeracao in enumerate(calculos_por_geracao.SigTipoGeracao):
        for nome_coluna in colunas_atrasos_medio:
            if nome_coluna == "Atraso_medio":
                continue
            count+=1
            try :
                ordered_notna = tempo_construcao_por_usina[(tempo_construcao_por_usina[nome_coluna].notna())
                                                 & (tempo_construcao_por_usina.SigTipoGeracao == SigTipoGeracao) ].sort_values(by = nome_coluna)
                x = ordered_notna[nome_coluna].dt.days.to_numpy().reshape(-1, 1)/30
                y = ordered_notna["Atraso_medio"].dt.days.to_numpy().reshape(-1, 1)/30
            
                res = stats.linregress(x.flatten(), y.flatten())
                a, b, r, p_value, std_err = res
                ax = axes[count,k]
                ax.scatter(x,y,color="blue")
                ax.set_xlabel(f"{nome_coluna} (meses)")
                ax.set_ylabel(f"Atraso_medio (meses)")
                ax.plot(x,a*x+b+1,color="red",label= f"{a:.1f}x+{b:.1f}")
                ax.set_title(f"\nTipo de geração: {SigTipoGeracao}   -  $R^2$ = {100*r**2:.1f}%\n Número de usinas: {x.shape[0]} \nOrigem: {origem}")
                ax.legend(loc="lower right")
                ax.grid()
            except (ValueError,AttributeError) as error:
                print(rf"Não foi possível calcular: {nome_coluna} para {SigTipoGeracao}")

plt.tight_layout()
plt.savefig("calculos_atrasos.pdf")